In [100]:
import pandas as pd
import urllib
import json


In [101]:
#Get series of stations
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190629.txt', usecols=['STATION'])
stations = df['STATION'].unique()


In [102]:
#Google maps url and api key for calling maps API
gm_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
api_key = 'Get your own!'


In [103]:
#Make df with stations as label, zip as column, use empty string for now
#This is what get turned into a csv
zip_codes = pd.DataFrame(data=stations, columns = ['STATION']).set_index('STATION')
zip_codes['ZIP'] = ''


In [104]:
for station in stations:    
    try:
        #Make url for calling GM API
        search = {'address': station + ' station, New York, NY', 'key' : api_key}
        url = gm_url + urllib.parse.urlencode(search)
        
        #Call the API and put into JSON
        data = urllib.request.urlopen(url).read()
        js = json.loads(data)
        
        #Select out the data we want  
        dicts = js['results'][0]['address_components'] #This is where an exception can happen
        zip_dict = {}
        for d in dicts:
            if d['types'] == ['postal_code']:
                zip_dict = d
                break
        zip_code = zip_dict['short_name']
        #Put zip codes in dataframe
        zip_codes.loc[station] = zip_code
    except:
        #Print out the stations we cant get zip codes for using the API, because of the way the name is formatted
        print(station)

CONEY IS-STILLW
HOWARD BCH JFK
MYRTLE AV
BROADWAY JCT
KEW GARDENS
JAMAICA CENTER
GRD CNTRL-42 ST
CENTRAL PK N110
NEREID AV
E 149 ST


In [105]:
#Manually fix stations we couldn't get a zip code for
manual_dict = {
'CONEY IS-STILLW': '11224',
'HOWARD BCH JFK':'11414' ,
'MYRTLE AV':'11206' ,
'BROADWAY JCT':'11207' ,
'KEW GARDENS': '11415',
'JAMAICA CENTER': '11432',
'GRD CNTRL-42 ST': '10017',
'CENTRAL PK N110':'10026' ,
'NEREID AV':'10470' ,
'E 149 ST': '10455'}
for station, zipcode in manual_dict.items():
    zip_codes.loc[station] = zip_code


In [106]:
zip_codes.rename(columns={'STATION':'station','ZIP':'zip_code'}, inplace=True)


In [107]:
zip_codes

,zip_code
STATION,
59 ST,10065
5 AV/59 ST,10022
57 ST-7 AV,10106
49 ST,10019
TIMES SQ-42 ST,10018
...,...
EASTCHSTER/DYRE,10466
ST. GEORGE,10301
TOMPKINSVILLE,10304


In [99]:
zip_codes.to_csv('zip_codes_new.csv', index=False)